In [170]:
import pandas as pd
import numpy as np
df = pd.read_csv('weather_classification_data.csv')
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [171]:
numercal_col = df.select_dtypes(include=['int64', 'number']).columns
numercal_col

Index(['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
       'Atmospheric Pressure', 'UV Index', 'Visibility (km)'],
      dtype='object')

In [172]:
df['Season'].unique()

array(['Winter', 'Spring', 'Summer', 'Autumn'], dtype=object)

In [173]:
categorical_col = df.select_dtypes(include=['object', 'category']).columns
categorical_col

Index(['Cloud Cover', 'Season', 'Location', 'Weather Type'], dtype='object')

In [174]:
X = df.drop('Weather Type', axis=1)
y = df['Weather Type']

In [175]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([1, 0, 3, ..., 0, 2, 1])

In [176]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [177]:
X_train.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location
3958,32.0,66,6.0,92.0,partly cloudy,1010.21,2,Spring,1.5,inland
2239,15.0,96,11.0,57.0,overcast,1016.98,0,Autumn,4.0,mountain
3608,28.0,58,6.5,11.0,clear,1025.37,10,Summer,9.0,inland
6848,47.0,84,44.5,85.0,overcast,992.26,2,Winter,2.0,coastal
6119,18.0,66,14.0,10.0,partly cloudy,1000.04,1,Autumn,8.0,coastal


In [178]:
df['Cloud Cover'].unique()

array(['partly cloudy', 'clear', 'overcast', 'cloudy'], dtype=object)

In [179]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
trf1 = make_column_transformer(
    (OneHotEncoder(drop='first'), [7, 9]),
    (OrdinalEncoder(categories='auto'), ['Cloud Cover']),
    remainder='passthrough'
)

In [180]:
X_train_transformed = trf1.fit_transform(X_train)
pd.DataFrame(X_train_transformed).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,0.0,0.0,1.0,0.0,3.0,32.0,66.0,6.0,92.0,1010.21,2.0,1.5
1,0.0,0.0,0.0,0.0,1.0,2.0,15.0,96.0,11.0,57.0,1016.98,0.0,4.0
2,0.0,1.0,0.0,1.0,0.0,0.0,28.0,58.0,6.5,11.0,1025.37,10.0,9.0
3,0.0,0.0,1.0,0.0,0.0,2.0,47.0,84.0,44.5,85.0,992.26,2.0,2.0
4,0.0,0.0,0.0,0.0,0.0,3.0,18.0,66.0,14.0,10.0,1000.04,1.0,8.0


In [182]:
from sklearn.preprocessing import StandardScaler
trf2 = make_column_transformer(
    (StandardScaler(), slice(0, 13)),
)

In [191]:
from sklearn.ensemble import RandomForestClassifier
trf3 = RandomForestClassifier()

In [192]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trf1, trf2, trf3)
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  [7, 9]),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Cloud Cover'])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  slice(0, 13, None))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [193]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  [7, 9]),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Cloud Cover'])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  slice(0, 13, None))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [194]:
pred = pipe.predict(X_test)

In [195]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9140151515151516